<a href="https://colab.research.google.com/github/elieserDev/plotly-ngrok-googlecolab/blob/master/Plotly_e_ngrok_Criando_um_servidor_com_acesso_externo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Aplicação Utilizando o Google Colab Com Plotly e ngrok

O verdadeiro motivo da existência de um Jupyter notebook, é para estudar, aprender e documentar processos desenvolvidos com Python.

Mas, estudando um pouco mais foi possível identificar algumas possibilidades avançadas com o Google Colab. A ferramenta já possibilita desenvolver funcionalidades no consumo de dados, com as demais ferramentas do Google, como: Google Documentos, Google Planilhas e etc.

Sendo assim, podemos identificar que o Google Colab foi desenvolvido com a intenção de análise e detalhamento de dados em cloud. Isso por meio de desenvolvimento de relatórios, utilizando Python, Javascript, HTML e outras mais linguagens suportadas pela plataforma.
Então, foi que percebi que poderia pensar um pouco fora da caixa... Porque não, pensar em algo um pouco mais avançado?

A ideia aqui, é criar um dash estático utilizando o Plotly, em seguida estruturar o ngrok para criar uma máscara e disponibilizar o nosso app para o mundo.
Mas, vamos tentar fazer uma simulação de liberação de permissão para os emails que desejamos apenas.

## Importando Bibliotecas
Aqui vamos importar todas as bibliotecas e módulos necessárias para trabalhar neste projeto.
Será necessário:

**Instalação das Bibliotecas Adicionais:**
- jupyter-dash
- pyngrok

**Importando Todas as Bibliotecas e Módulos:**
- requests(Trabalhando com solicitações HTTP)
- auth(Módulo Google Colab para trabalhar com autenticação)
- px(Módulo Plotly para trabalhar com Plotly Express)
- dcc(Módulo dash para podermos trabalhar com os componentes do painel final)
- html(Modulo dash para trabalhar com os elementos HTML da página final do painel)
- Input e Output(Modulos dash para trabalhar com os callbacks de entrada e saída do app)
- ngrok(Biblioteca para trabalharmos com o nosso backdoor)
- IPython(Biblioteca para trabalharmos a integração entre o Python e o navegador local)

In [ ]:
#Instalando pacotes:
!pip install jupyter-dash
!pip install pyngrok
#Importação de Libs

import requests
from google.colab import auth
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output# Load Data
from pyngrok import ngrok
import IPython

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 81kB 3.0MB/s 
     |████████████████████████████████| 1.0MB 17.3MB/s 
     |████████████████████████████████| 3.5MB 33.4MB/s 
     |████████████████████████████████| 194kB 51.1MB/s 
     |████████████████████████████████| 1.8MB 40.9MB/s 
     |████████████████████████████████| 358kB 44.3MB/s 
  Created wheel for dash: filename=dash-1.18.1-cp36-none-any.whl size=83681 sha256=3933d4e867ca960b861778a9f01ec25b824cc5cc053a1e80d772f081ad1b0488
  Stored in directory: /root/.cache/pip/wheels/eb/56/fb/79b2169ce9fcb79753ec57a16abb8f0b7750b4c63d7eb3cea9
  Created wheel for dash-renderer: filename=dash_renderer-1.8.3-cp36-none-any.whl size=1013945 sha256=46fea9e99b90ab6c0b3e80ba1524c9fa30e83287bafdb0d6a5cb504aa02b350f
  Stored in directory: /root/.cache/pip/wheels/cf/2b/5f/0928a6d1b7ebf280f21a2e925f36d662e6ba83e00b82c6b6bf
  Created wheel for dash-core-components: filename=dash_core_components-1.14.1-cp36

#Desenvolvendo um controle de acesso por autenticação
Após termos baixado, instalado e importado tudo que precisávamos. Agora, vamos trabalhar com o auth do Google Colab para possibilitar o usuário logar com a sua conta do Google, neste notebook.

Assim, logo podemos extrair o email fornecido pelo login e verificar se o seu email está dentro da lista de permissões ou não.

Caso esteja, o acesso é concedido. Caso não, o acesso é negado.

In [ ]:
#Autenticação:
auth.authenticate_user()
gcloud_token = !gcloud auth print-access-token
google = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
# !gcloud config get-value account

permissao = [
  'elieser.oliveira@compasso.com.br',
]

def validacao():
  return google['email'] in permissao

## Criando e Estruturando o APP

Neste momento, vamos ver como desenvolver os controles da aplicação e também como levantar o servidor e backdoor usando o Plotly e o ngrok.

Vamos utilizar o Python para desenvolver o back-end e Javascript, para trabalhar com os mecanismos do navegador.

Com o Javascript, vamos desenvolver o controle de estabilização do notebook, acesso automático ao app na primeira vez que rodar todo o notebook e a criação de botões para acessar e desabilitar o monitoramento do terminal de uso deste notebook.

### Primeira Célula Deste Processo

Na próxima célula, vamos cuidar deste processo de monitoramento do nosso front-end, ou seja, do nosso notebook como um todo.

Precisamos passar o valor que retorna da função em Python que se responsabiliza pela validação de acesso ao app. Para isso, precisaremos usar o IPython, onde iremos criar uma string com a sintaxe da criação de uma variável em Javascript.
Assim, poderemos criar uma variável global, onde será armazenado o valor dinâmico de validação. Isso porque não conseguimos passar um valor que resulta do contexto Python, para um contexto Javascript. Até por se tratarem de ambientes e contextos diferentes.

Sendo assim, usaremos a função lower() do Python para que o resultado esteja adaptado para a sintaxe Javascript. Se o valor retornado da função validacao() que fica no contexto Python retorna True, então o valor que será armazenado na variável global no Javascript será true. Se o valor retornado da função validacao() que fica no contexto Python retorna False, então o valor que será armazenado na variável global no Javascript será false. Respeitando o lower case da sintaxe Javascript.

A linha onde será armazenado o valor no formato de string, dentro do Python, será representada da seguinte forma:


```python
  var_js = 'var validacao = ' + str(validacao()).lower() #Passando o valor para JS
```
Assim, usaremos o display para passar este contexto para Javascript usando o IPython, na próxima linha da seguinte forma:
```python
  display(IPython.display.Javascript(var_js)) #Enviando variável para contexto JS
```
No final das "contas", o que esperamos que aconteça no "lado" do Javascript, é que apenas seja passado a string como um todo. Além do mais, o que estamos passando pelo IPython é o valor da variável __var_js__.

Então, o resultado esperado que seja passado para o contexto Javascript, será o seguinte:
```javascript
  var validacao = true
```
_*ou*_
```javascript
  var validacao = false
```

Essa lógica estará dentro de uma função Python, chamada cria_controle_app(), logo nas primeiras linhas.

Dentro desta função, também é encontrado um outro display, que servirá para escrever o nosso código Javascript, responsável por manter o nosso notebook conectado ao servidor. Do contrário, após 30 minutos o notebook é desconectado automaticamente, por inatividade. Além disso, este código Javascript servirá para criar alguns botões interativos.

In [ ]:
#Criando controle de botões e acesso automático ao app
def cria_controle_app():
  var_js = 'var validacao = ' + str(validacao()).lower() #Passando o valor para JS
  display(IPython.display.Javascript(var_js)) #Enviando variável para contexto JS
  
  display(IPython.display.Javascript("""
    function RunApp() {
      var urlApp = document.querySelector('#output-body .output-id-2 a');
      console.log(urlApp);
      if (urlApp) window.open(urlApp.href);
      
      //Criando Botão do App
      var btn = document.createElement('button');
      var texto_btn = document.createTextNode('Abrir App');
      var link = document.createElement('a');
      link.setAttribute('href', urlApp);
      link.setAttribute('target', '_blank');
      link.setAttribute('id', 'server_url');
      btn.appendChild(texto_btn);
      document.querySelector('#output-area').appendChild(link);
      document.querySelector('#server_url').appendChild(btn);

      //Criando Botão de cancelar o interval
      CriaBtnControle();
    }

    function CriaBtnControle() {
      var btn_controle = document.createElement('button');
      var texto_controle = document.createTextNode('Parar Controlador');
      
      btn_controle.setAttribute('id', 'estabilizador');
      btn_controle.setAttribute('type', 'button');
      btn_controle.setAttribute('onclick', 'Controlador()');
      btn_controle.appendChild(texto_controle);
      document.querySelector('#output-area').appendChild(btn_controle);
    }

    function Conectar() {
      console.log('Conectado!');
      window.location.reload();
    }

    function Controlador() {
      clearInterval(loop);
      console.warn('Controlador de conexão finalizado!')
    }

    if (validacao) {
      setTimeout(RunApp, 3000); 
      var loop = setInterval(Conectar, 60000);
    }
  """))

### Validando, Criando Servidor e Disponibilisando o App

Aqui inicia o desenvolvimento do relatório com Plotly, usando ngrok para backdoor. Assim, criando um túnel para a aplicação dentro do servidor que ficará hospedada na máquina da Google. Gerando um link com disponibilidade de acesso externo.

logo no início criamos um dicionário com duas chaves, uma mensagem de sucesso e outra de falha. Cada uma com um código de estilo: Azul para sucesso e vermelho para falha.

Aqui será criado uma tela bem simples, com dados estáticos e um dash "aleatório". Apenas, para simplificar o método didático.

In [ ]:
mensagem = {
  'sucesso' : '\033[0;34mAcesso concedido!\033[m',
  'falha' : '\033[0;31;47mVocê não tem acesso ao relatório completo!\033[m'
}

if validacao():
  print(mensagem['sucesso'])
  df = px.data.tips()# Build App
  app = JupyterDash(__name__)
  app.layout = html.Div([
      html.H1("1littlecoder deploy ML App Colab ngrok Demo"),
      dcc.Graph(id='graph'),
      html.Label([
          "colorscale",
          dcc.Dropdown(
              id='colorscale-dropdown', clearable=False,
              value='plasma', options=[
                  {'label': c, 'value': c}
                  for c in px.colors.named_colorscales()
              ])
      ]),
  ])# Define callback to update graph
  @app.callback(
      Output('graph', 'figure'),
      [Input("colorscale-dropdown", "value")]
  )
  def update_figure(colorscale):
      return px.scatter(
          df, x="total_bill", y="tip", color="size",
          color_continuous_scale=colorscale,
          render_mode="webgl", title="Tips"
      )# Executa o app e exibe o resultado no notebook
  app.run_server(mode='external')
  # Abre um tunnel HTTP na porta 80 como padrão
  public_url = ngrok.connect(port = '8050')
else:
  print(mensagem['falha'])

# Criando controles e disponibilizando App em uma nova aba.
cria_controle_app()

Acesso concedido!
Dash app running on:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>